<a href="https://colab.research.google.com/github/avrymi-asraf/IDL-huji/blob/main/ex1/ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch

In [ ]:
# # for clab
# !git clone https://github.com/avrymi-asraf/IDL-huji.git
# !mv /content/IDL-huji/ex1/ex1_data* .